In [1]:
import os
import numpy as np
from scipy.integrate import odeint
import netwin as nw
import torch 
import sbi.utils as utils
from sbi.inference.base import infer

In [2]:
root_dir = '/home/chaggar/Documents/Network_Inference'
graph_path = os.path.join(root_dir, 'data/brain_networks/scale1.csv')

In [3]:
class NetworkDiffusion(nw.Model):

    def f(self, p, t, theta):
        k = theta
        du = k * (-self.L() @ p)
        return du

    def solve(self, p, theta):
        return odeint(self.f, p, self.t, args=(theta,))

    def forward(self, u0): 
        p = np.exp(u0[:-1])
        theta = u0[-1]
        
        u = self.solve(p,theta) 
        return u.flatten()

In [4]:
m = NetworkDiffusion(graph_path)

In [5]:
p = np.ones([83])
p[40] = 10
k = 5

u0 = np.append(np.log(p), k)

m.t = np.linspace(0,0.1,100)

In [6]:
sim = m.f(p = p, t = m.t, theta = k)

In [7]:
data = m.forward(u0=u0)

In [8]:
dims = 84
prior = utils.BoxUniform(low=0*torch.ones(dims), high=20*torch.ones(dims))

In [10]:
posterior = infer(m.forward, prior, 'SNPE', num_simulations=10, num_workers=4)


Neural network successfully converged after 112 epochs.


KeyboardInterrupt: 

In [ ]:
posterior_samples = posterior.sample((10000,), x=data)